In [1]:
import pandas as pd
from cddd.inference import InferenceModel
from cddd.preprocessing import preprocess_smiles
from scipy.stats import norm
import os

In [2]:
import numpy as np
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from IPython.display import SVG
from scipy import stats

In [3]:
from rdkit.Chem import AllChem as Chem

In [4]:
inference_model = InferenceModel('/home/DeepLearning/cddd/default_model/', 
                                 use_gpu=False, beam_width=10, num_top=3)

In [5]:
encoder = inference_model.seq_to_emb
decoder = inference_model.emb_to_seq

### Prepare the data

In [6]:
data_dir = '/home/Development/GAN_testing/A2AR'
data_file = 'A2AR.csv'
num_classes = 2

In [7]:
csv_file = os.path.join(data_dir, data_file)
init_data = pd.read_csv(csv_file)

In [8]:
init_data.sample(5)

,SMILES,label,CMPD_CHEMBLID
627,CCCCC(=O)N1CCN(c2nc(N)n3nc(-c4ccco4)nc3n2)CC1,1,CHEMBL1927435
3556,Nc1nc(-c2cccc(Cl)c2)nc2sc(CN3CCOCC3)cc12,1,CHEMBL3222105
3141,Cn1c2ccccc2n2c(=O)c(-c3ccco3)nnc12,0,CHEMBL7098
420,CC(c1cc2ccccc2s1)N(O)C(N)=O,0,CHEMBL93
912,CCCn1c(=O)c2[nH]c(-c3ccc(OCC(=O)Nc4ccc(C(N)=O)...,1,CHEMBL201791


In [10]:
X = encoder(init_data.SMILES.values)
ys = init_data.label.values

In [11]:
assert(len(X) == len(ys))
X_file = os.path.join(data_dir, 'X_act.npy')
Y_file = os.path.join(data_dir, 'Y_act.npy')
np.save(X_file, X)
np.save(Y_file, ys)

### GAN using pytorch
#### https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/wgan_gp/wgan_gp.py


In [12]:
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
import os
import torchvision.transforms as transforms

from torch.autograd import Variable

import torch.autograd as autograd
import torch.nn.init as init

In [13]:
lr = 0.0001
b1 = 0.0 # 0.5
b2 = 0.9 #0.999
batch_size = 32
epochs = 10000
latent_dim =512
input_dim = 32
# WGAN-GP
n_critic = 5   # number of training steps for discriminator per iter
acgan_scale = 1
acgan_scale_G = 1
# Loss weight for gradient penalty , default 10
lambda_gp = 10

In [14]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        if m.weight is not None:
            init.xavier_uniform_(m.weight)
        if m.bias is not None:
            init.constant_(m.bias, 0.0)

In [15]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.block = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.4),
            nn.Linear(128, 64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.4)
        )
        self.adv_layer = nn.Linear(64, 1)
        self.aux_layer = nn.Sequential(nn.Linear(64, num_classes), nn.Softmax()) #nn.Linear(64, num_classes)

    def forward(self, molvec):
        # Concatenate label and mol to produce input
#         dis_input = torch.cat((molvec, labels), -1)
        out = self.block(molvec)
        validity = self.adv_layer(out)
        label = self.aux_layer(out)
        return validity, label

In [16]:
# keep close to Bayer cddd paper dim=1024 for the input
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        self.label_emb = nn.Embedding(num_classes, input_dim)

        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 2048),
            nn.BatchNorm1d(2048),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(2048, latent_dim),
            nn.Tanh()
        )

    def forward(self, z, labels):
        # Concatenate label and mol to produce input
        gen_input = torch.mul(self.label_emb(labels), z)#torch.cat((labels, z), -1)
        return self.model(gen_input)

In [17]:
# Loss functions
auxiliary_loss = torch.nn.CrossEntropyLoss()

In [18]:
# initialize G, D
gen = Generator()
dis = Discriminator()
if torch.cuda.is_available():
    gen.cuda()
    dis.cuda()
    auxiliary_loss.cuda()
gen.apply(weights_init)
dis.apply(weights_init)

Discriminator(
  (block): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): LeakyReLU(negative_slope=0.2, inplace)
    (4): Dropout(p=0.4)
    (5): Linear(in_features=128, out_features=64, bias=True)
    (6): LeakyReLU(negative_slope=0.2, inplace)
    (7): Dropout(p=0.4)
  )
  (adv_layer): Linear(in_features=64, out_features=1, bias=True)
  (aux_layer): Sequential(
    (0): Linear(in_features=64, out_features=2, bias=True)
    (1): Softmax()
  )
)

In [19]:
# Optimizers
optimizer_G = torch.optim.Adam(gen.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(dis.parameters(), lr=lr, betas=(b1, b2))

In [20]:
# Configure data loader
class LabeledMolvecs(torch.utils.data.Dataset):
    def __init__(self, X_file, Y_file, data_dir, transform=None):
        self.X = np.load(os.path.join(data_dir, X_file))
        self.Y = np.load(os.path.join(data_dir, Y_file))
        self.transform = transform
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, index):
        return (self.X[index, :], self.Y[index])  #.reshape(-1, 1)


In [21]:
dataloader = torch.utils.data.DataLoader(
    LabeledMolvecs('X_act.npy', 'Y_act.npy', data_dir),
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

In [22]:
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if torch.cuda.is_available() else torch.LongTensor

In [23]:
def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1)))
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates, _ = D(interpolates)
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [24]:
##########
# TRAINING
##########

batches_done = 0
for epoch in range(epochs):
    for i, (molvecs, labels) in enumerate(dataloader):
        # Configure input
        real_mols = Variable(molvecs.type(Tensor))
        labels = Variable(labels.type(LongTensor))
        
        #####################
        # Train Discriminator
        #####################
        
        optimizer_D.zero_grad()
        
        # Sample noise and labels as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (molvecs.shape[0], input_dim))))
        f_labels = np.random.randint(0, num_classes, molvecs.shape[0])
        fake_labels = Variable(LongTensor(f_labels))
        
        # Generate molecules
        fake_mols = gen(z, fake_labels)
        
        # Real mols
        real_validity, real_aux = dis(real_mols)
        real_err_D = torch.mean(real_validity)
        real_aux_err_D = auxiliary_loss(real_aux, labels)#.mean()
        
        # Fake mols
        fake_validity, fake_aux = dis(fake_mols)
        fake_err_D = torch.mean(fake_validity)
        
        # Gradient penalty
        gradient_penalty = compute_gradient_penalty(dis, real_mols.data, fake_mols.data)
        # Adversarial loss
        cost_D = -real_err_D + fake_err_D + lambda_gp * gradient_penalty
        acgan_D = real_aux_err_D
        
        d_loss = cost_D + acgan_scale*acgan_D
        
        # Calculate discriminator accuracy
        pred = np.concatenate([real_aux.data.cpu().numpy(), fake_aux.data.cpu().numpy()], axis=0)
        gt = np.concatenate([labels.data.cpu().numpy(), fake_labels.data.cpu().numpy()], axis=0)
        d_acc = np.mean(np.argmax(pred, axis=1) == gt)
        
        d_loss.backward()
        optimizer_D.step()
        
        
        #################
        # Train Generator
        #################
        
        optimizer_G.zero_grad()
        
        # Train the generator every n_critic steps
        if i % n_critic == 0:
            # Sample noise and labels as generator input
            z = Variable(Tensor(np.random.normal(0, 1, (molvecs.shape[0], input_dim))))
            f_labels = np.random.randint(0, num_classes, molvecs.shape[0])
            fake_labels = Variable(LongTensor(f_labels))
            
            # Generate molecules
            fake_mols = gen(z, fake_labels)
            # Loss measures generator's ability to fool the discriminator
            # Train on fake mols
            fake_validity, pred_label = dis(fake_mols)
#             f_labels = Variable(LongTensor(f_labels))
            aux_loss = auxiliary_loss(pred_label, fake_labels)#.mean()
            gen_cost = -torch.mean(fake_validity)
            
            g_loss = acgan_scale_G*aux_loss + gen_cost
            
            g_loss.backward()
            optimizer_G.step()
        
            ##############################
        
            print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f] [WD: %f] [D acc: %f]"
            % (epoch, epochs, i, len(dataloader), d_loss.item(), g_loss.item(), 
               -real_err_D.item() + fake_err_D.item(), 100*d_acc)
            )
        
            batches_done += n_critic
        

/home/anaconda3/envs/cddd/lib/python3.6/site-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


[Epoch 0/10000] [Batch 0/145] [D loss: 2.438668] [G loss: 0.339236] [WD: -0.474179] [D acc: 46.875000]
[Epoch 0/10000] [Batch 5/145] [D loss: 2.071541] [G loss: 0.137721] [WD: -0.271429] [D acc: 57.812500]
[Epoch 0/10000] [Batch 10/145] [D loss: 1.616674] [G loss: -0.034660] [WD: -0.171434] [D acc: 65.625000]
[Epoch 0/10000] [Batch 15/145] [D loss: 1.756667] [G loss: 0.145568] [WD: -0.434483] [D acc: 39.062500]
[Epoch 0/10000] [Batch 20/145] [D loss: 1.456950] [G loss: 0.113352] [WD: -0.453810] [D acc: 60.937500]
[Epoch 0/10000] [Batch 25/145] [D loss: 1.319772] [G loss: 0.065557] [WD: -0.642230] [D acc: 54.687500]
[Epoch 0/10000] [Batch 30/145] [D loss: 0.944933] [G loss: 0.044022] [WD: -0.732060] [D acc: 57.812500]
[Epoch 0/10000] [Batch 35/145] [D loss: 0.773891] [G loss: -0.050153] [WD: -0.948444] [D acc: 45.312500]
[Epoch 0/10000] [Batch 40/145] [D loss: 0.819515] [G loss: -0.091972] [WD: -0.886656] [D acc: 48.437500]
[Epoch 0/10000] [Batch 45/145] [D loss: 0.955783] [G loss: 0.18

[Epoch 9997/10000] [Batch 55/145] [D loss: -0.150480] [G loss: -0.091930] [WD: -0.857152] [D acc: 92.187500]
[Epoch 9997/10000] [Batch 60/145] [D loss: -0.228761] [G loss: -0.145526] [WD: -1.051074] [D acc: 81.250000]
[Epoch 9997/10000] [Batch 65/145] [D loss: -0.220584] [G loss: -0.142666] [WD: -0.943862] [D acc: 85.937500]
[Epoch 9997/10000] [Batch 70/145] [D loss: -0.125907] [G loss: -0.216884] [WD: -0.740148] [D acc: 90.625000]
[Epoch 9997/10000] [Batch 75/145] [D loss: -0.286333] [G loss: -0.350579] [WD: -0.864278] [D acc: 89.062500]
[Epoch 9997/10000] [Batch 80/145] [D loss: -0.333137] [G loss: -0.022999] [WD: -1.064658] [D acc: 93.750000]
[Epoch 9997/10000] [Batch 85/145] [D loss: -0.213069] [G loss: -0.053483] [WD: -0.783357] [D acc: 87.500000]
[Epoch 9997/10000] [Batch 90/145] [D loss: -0.304944] [G loss: -0.234313] [WD: -0.957269] [D acc: 82.812500]
[Epoch 9997/10000] [Batch 95/145] [D loss: -0.258049] [G loss: 0.053753] [WD: -0.895224] [D acc: 87.500000]
[Epoch 9997/10000] [

###  Save the DNN model

In [25]:
torch.save(gen, os.path.join(data_dir, 'generator.pt'))
torch.save(dis, os.path.join(data_dir, 'discriminator.pt'))

/home/anaconda3/envs/cddd/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Generator. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/anaconda3/envs/cddd/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Discriminator. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


## Generate molecules

In [26]:
# Sample
def sample(gen, dis, batch_size, labels):
    # set model in eval mode
    gen.eval()
    
    def mol_from_xsmiles(smis):
        if isinstance(smis, str):
            return Chem.MolFromSmiles(smis)
        for smi in smis:
            mol = Chem.MolFromSmiles(smi)
            if mol is not None:
                return mol
        return None
    
    with torch.no_grad():
        z = Variable(Tensor(np.random.normal(0, 1, (batch_size, input_dim))))
        labels = Variable(LongTensor(labels))
    
    # create a sample
    x = gen(z, labels)
#     __, D_labels = dis(x)
    
    x = x.cpu().detach().numpy()
#     D_labels = D_labels.cpu().detach().numpy()
#     print(np.argmax(D_labels, axis=1))
    Y = decoder(x)
    mols = [mol_from_xsmiles(y) for y in Y]
    return mols

In [27]:
g_mols = sample(gen, dis, batch_size, np.ones(batch_size, dtype=int)*1)

INFO:tensorflow:Restoring parameters from /home/DeepLearning/cddd/default_model/model.ckpt


RDKit ERROR: [18:38:22] Can't kekulize mol.  Unkekulized atoms: 1 2 3 9 10 11 12 20 21
RDKit ERROR: 
RDKit ERROR: [18:38:22] Can't kekulize mol.  Unkekulized atoms: 4 5 6 7 8 20 21 27 28 33
RDKit ERROR: 
RDKit ERROR: [18:38:22] Can't kekulize mol.  Unkekulized atoms: 4 5 6 7 8 20 21 27 28 33
RDKit ERROR: 
RDKit ERROR: [18:38:22] Can't kekulize mol.  Unkekulized atoms: 4 5 6 7 8 20 21 27 28
RDKit ERROR: 
RDKit ERROR: [18:38:22] Can't kekulize mol.  Unkekulized atoms: 3 4 5 12 19
RDKit ERROR: 
RDKit ERROR: [18:38:22] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 15 22 23
RDKit ERROR: 
RDKit ERROR: [18:38:22] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 15 22 23
RDKit ERROR: 
RDKit ERROR: [18:38:22] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 15 22 23
RDKit ERROR: 


### Load QSAR model

In [28]:
import sklearn
from sklearn.externals import joblib
print(sklearn.__version__)

from rdkit.Chem import AllChem as Chem
from rdkit.Chem import DataStructs, Descriptors

0.20.1


In [29]:
def morganfp(mol, bits=4096, radius=3):
    if mol is None: return
    vec = np.ndarray((1, bits), dtype=int)
    fp = Chem.GetMorganFingerprintAsBitVect(mol, radius, nBits=bits)
    DataStructs.ConvertToNumpyArray(fp, vec)
    return vec

In [30]:
filename = os.path.join(data_dir, 'A2AR.jbl')
model = joblib.load(filename)

In [31]:
qsar_vec = []
good_mols=[]
idx = []
for i, m in enumerate(g_mols):
    fp = morganfp(m)
    if fp is not None:
        qsar_vec.append(fp)
        good_mols.append(m)
    else:
        idx.append(i)

In [32]:
preds = model.predict(qsar_vec)#.tolist()

In [33]:
len(preds)

30

In [ ]:
Chem.Draw.MolsToGridImage(g_mols, molsPerRow=2, maxMols=200, subImgSize=(600, 600),)

In [37]:
## But how many are there in the training set?
good_smiles = list(Chem.MolToSmiles(x) for x in good_mols)
count = 0
new = 0
for i, smi in enumerate(good_smiles):
    count += 1
    if smi in init_data.SMILES.tolist():
        print(smi, preds[i])
    else:
        new += 1

print('Ratio of new: {:.2f}'.format(new / count))
mean_act = np.mean(preds)
print('Mean activity {:.2f}'.format(mean_act))

O=C(Nc1ccc(-c2ccncc2)c(-c2ccccc2F)n1)C1CC1 1
Cn1cc2c(nc(NC(=O)Cc3ccc(C(F)(F)F)cc3)n3nc(-c4ccco4)nc23)n1 1
Nc1nc(-c2ccco2)nc2sc(CN3CCSCC3)cc12 1
Nc1nc(-c2ccccc2)nc2sc(Cc3ccccc3)cc12 1
Ratio of new: 0.87
Mean activity 0.80


### Tougher test

In [38]:
n_samples = 10000 // batch_size

In [39]:
n_samples

312

In [40]:
LABEL = 1

In [41]:
def unique(mols):
    smi = [Chem.MolToSmiles(x) for x in mols if x is not None and Chem.MolToSmiles(x) is not None]
    smi = list(set(smi))
    ret = [Chem.MolFromSmiles(x) for x in smi if Chem.MolFromSmiles(x) is not None]
    return ret

In [ ]:
g_mols_tough = []
for i in range(n_samples):
    g_mols_tough.append(sample(gen, dis, batch_size, np.ones(batch_size, dtype=int)*LABEL))

In [43]:
from functools import reduce

In [44]:
g_mols_tough = list(reduce(lambda x,y: x+y, g_mols_tough))

In [45]:
len(g_mols_tough)

9984

In [ ]:
unique_mols = unique(g_mols_tough)

In [48]:
qsar_vec_tough = []
good_mols_tough=[]
idx = []
for i, m in enumerate(unique_mols):
    fp = morganfp(m)
    if fp is not None:
        qsar_vec_tough.append(fp)
        good_mols_tough.append(m)
    else:
        idx.append(i)

In [49]:
preds_tough = model.predict(qsar_vec_tough)

In [50]:
np.sum(preds_tough == 1) / len(preds_tough)

0.63467139420032925

In [51]:
len(unique_mols)

7897

In [52]:
## But how many are there in the training set?
good_smiles_tough = list(Chem.MolToSmiles(x) for x in unique_mols)
count = 0
new = 0
recovered = 0
for i, smi in enumerate(good_smiles_tough):
    count += 1
    if smi in init_data.SMILES.tolist():
        print(smi, preds_tough[i])
    else:
        new += 1


Nc1nc(-c2ccccc2)nc2cn(Cc3ccccc3)nc12 1
CCC(=O)Nc1cc(-c2ccco2)nc(-c2ccco2)n1 1
Nc1nc(-c2ccco2)c2cnn(Cc3ccccc3)c2n1 1
Nc1nc(-c2ccccc2)cc(-c2ccccc2)n1 0
Nc1nc(C(=O)NCc2cccc3cccnc23)c2ccccc2n1 1
COc1ccc(Cn2cnc3c(-c4ccco4)nc(N)nc32)cc1 1
CCn1cc2c(nc(NC(=O)Nc3ccc(F)cc3)n3nc(-c4ccco4)nc23)n1 1
Nc1nc(-c2ccc(Br)o2)nc2sc(CN3CCCC3)cc12 1
Cn1cc2c(nc(NC3CCCC3)n3nc(-c4ccco4)nc23)n1 1
CCCn1c(=O)c2[nH]c(-c3ccc(OCC(=O)NCc4ccccc4)cc3)nc2n(CCC)c1=O 1
Cn1cc2c(nc(NC(=O)Nc3ccc(C(F)(F)F)cc3)n3nc(-c4ccco4)nc23)n1 1
Nc1nc2c(cnn2CCc2ccc(OCc3ccccc3)cc2)c2nc(-c3ccco3)nn12 1
Nc1nc(-c2ccco2)c2cnn(Cc3ccccc3[N+](=O)[O-])c2n1 1
Nc1nc(C(=O)NCc2ccco2)cc(-c2ccco2)n1 1
O=C(Nc1ccc(-c2ccncc2F)c(-c2ccco2)n1)C1CC1 1
Cc1nc(-c2sc(NC(=O)c3ccccc3)nc2-c2ccccc2)no1 1
Cc1ccc(-c2cc(-c3ccccc3)nc(N)c2C#N)cc1 1
CCCC(=O)Nc1cc(-c2ccccc2)nc(-c2ccccc2)n1 1
Cc1cccc(-c2cc(C(=O)NCc3ccccn3)nc(N)n2)c1 1
Nc1nc(-c2ccco2)c2ncn(Cc3cccc(Cl)c3)c2n1 1
O=C(Nc1cnc(-c2ccncc2)c(-c2ccco2)n1)C1CC1 1
Cn1c(=O)c2[nH]c(-c3ccc(OCC(=O)N4CCc5ccccc5C4)cc3)cc2n(C)c1=

O=C(Nc1cc(-c2ccccc2)nc(-c2ccccc2)n1)C1CCCC1 1
CCCCc1nc2n[nH]cc2c2nc(-c3ccccc3)nn12 1
COc1ccccc1-c1cc(C(=O)NCc2ncccc2C)nc(N)n1 1
CCCn1c(=O)c2[nH]c(-c3ccc(OCC(=O)Nc4ccc(Cl)cc4)cc3)cc2n(CCC)c1=O 1
O=C(Nc1ccccc1)Nc1nc(NC2CCCCC2)nc2nc(-c3ccco3)nn12 1
Uniqueness ratio: 0.79
Ratio of new: 0.97


In [264]:
recovered

11

In [152]:
idx = preds_tough == 1
sel_mols_tough = np.asarray(good_mols_tough)[idx]
sel_preds_tough = np.asarray(preds_tough)[idx]

In [53]:
df = pd.DataFrame({'SMILES': good_smiles_tough, 'label': preds_tough})

In [55]:
df.to_csv(os.path.join(data_dir, 'ACGAN_A2AR_{}_epochs.csv'.format(epochs)))